<a href="https://www.kaggle.com/code/baongocdao/text-summarization?scriptVersionId=194760025" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.4 MB/s eta 0:00:00a 0:00:01m


In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 836.5 kB/s eta 0:00:000:00:01


In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4573a791cb25f750946da6da4eda6d6f622a29bbfc06c6449ee87323155be414
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from peft import PrefixTuningConfig, get_peft_model, TaskType
import torch
from datasets import load_dataset
import re
from torch.utils.data import DataLoader
import evaluate

rouge_score = evaluate.load('rouge')
tokenizer = T5TokenizerFast.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [6]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [7]:
def remove_tag(text):
    ending_phrase = 'E-mail to a friend'
    title_phrase = '--'
   
    title_idx = text.find(title_phrase)
    end_idx = text.find(ending_phrase) 
    
    if end_idx != -1 :
        text = text[:end_idx]
    if title_idx != -1 :
        text = text[title_idx+3 :]
    text = re.sub(r'/(CNN/)', '', text)
    return text.strip()

def remove_hashtags(text):
    cleaned_text = re.sub(r'#\w+', '', text)
    return cleaned_text.strip()

def remove_newline(text):
    cleaned_text = re.sub(r'\n', '', text)
    return cleaned_text.strip()
def filter_chars(text):
    sent = []
    for word in text.split(' '):
        if ('$' in word) or ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def clean_data(text):
    text = remove_tag(text)
    text = remove_hashtags(text)
    text = filter_chars(text)
    text = remove_newline(text)
    return text

def preprocess(example):
    text = example['article']
    summary = example['highlights']
    
    text = clean_data(text)
    summary = clean_data(text)
    
    text_token = tokenizer(text, padding = 'max_length')
    summary_token = tokenizer(summary, padding = 'max_length')
    return {
        'text_input_ids': text_token['input_ids'],
        'attention_mask': text_token['attention_mask'],
        'summary_input_ids': summary_token['input_ids']
    }

In [8]:
train_dt = dataset['train'].select(range(10000))
valid_dt = dataset['validation'].select(range(2000))

train_set = train_dt.map(preprocess)
valid_set = valid_dt.map(preprocess)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token

In [10]:
def get_collate(pad_idx):
    def collate_fn(batch):
        text_input_ids = [torch.tensor(i['text_input_ids']) for i in batch]
        text_input_ids = torch.nn.utils.rnn.pad_sequence(text_input_ids, 
                                                      padding_value = pad_idx, 
                                                      batch_first = True)
        summary_input_ids = [torch.tensor(i['summary_input_ids']) for i in batch]
        summary_input_ids = torch.nn.utils.rnn.pad_sequence(summary_input_ids,
                                                        padding_value = pad_idx,
                                                        batch_first = True)
        attention_mask = [torch.tensor(i['attention_mask']) for i in batch]
        attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, 
                                                      padding_value = pad_idx,
                                                      batch_first = True)
        
        return {
                'text_input_ids': text_input_ids,
                'attention_mask': attention_mask,
                'summary_input_ids': summary_input_ids
            }      
    return collate_fn

In [11]:
def get_dataloader(dataset, pad_idx, batch_size):
    collate_fn = get_collate(pad_idx)
    return DataLoader(dataset, 
                      batch_size = batch_size, 
                      collate_fn = collate_fn, 
                      shuffle = True)

In [12]:
batch_size = 2
pad_idx = tokenizer.pad_token_id
train_dataset = get_dataloader(train_set, pad_idx = pad_idx, batch_size = batch_size)
valid_dataset = get_dataloader(valid_set, pad_idx = pad_idx, batch_size = batch_size)

In [13]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, 
                                 inference_mode=False, 
                                 num_virtual_tokens=20)

origin_model = T5ForConditionalGeneration.from_pretrained('t5-small')
model = get_peft_model(origin_model, peft_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 122,880 || all params: 60,629,504 || trainable%: 0.2027


In [14]:
model

PeftModelForSeq2SeqLM(
  (base_model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Lin

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)
model = torch.nn.DataParallel(model)

In [16]:
import tqdm
def train_session(model, dataloader, optimizer, device):
    epoch_loss = []
    model.train()
    for batch in tqdm.tqdm(dataloader, desc = 'training....'):
        input_ids = batch['text_input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['summary_input_ids'].to(device)
        output = model(input_ids = input_ids, 
                       attention_mask = attention_mask, 
                       labels = labels)
        loss = output.loss.mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss.append(loss.item())
        del batch
        del loss

    return np.mean(epoch_loss)

In [17]:
def compute_rouge(predict_ids, labels):
    references = tokenizer.batch_decode(labels)
    predictions = tokenizer.batch_decode(predict_ids)
    score = rouge_score.compute(predictions = predictions, references = references)
    return score['rougeLsum']
    
def eval_session(model, dataloader, optimizer, device):
    epoch_loss = []
    epoch_rouge = []
    model.eval()
    for batch in tqdm.tqdm(dataloader, desc='evaluate.....'):
        input_ids = batch['text_input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['summary_input_ids'].to(device)
        with torch.no_grad():
            original_model = model.module
            output = model(input_ids = input_ids, 
                           attention_mask = attention_mask,
                           labels = labels)
            predict_ids = original_model.generate(input_ids = input_ids,
                                                  num_beams = 5,
                                                  early_stopping = True,
                                                  eos_token_id=tokenizer.eos_token_id,
                                                  do_sample = True,
                                                  top_p = 0.9, 
                                                  top_k = 40,
                                                  max_length = 180)
            loss = output.loss.mean()
            epoch_loss.append(loss.item())
            rouge = compute_rouge(predict_ids, labels)
            epoch_rouge.append(rouge)
    return {
        'loss': np.mean(epoch_loss),
        'rouge': np.mean(epoch_rouge)
            }

In [18]:
lr = 2e-4
optimizer = torch.optim.AdamW(model.parameters(), lr= lr, weight_decay = 1e-5)
epoch = 5
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma=0.5)
loss, loss_eval = [], []
rouge_epoch = []
for i in range(epoch):
    print(f'epoch:{i}')
    
    loss_value = train_session(model, train_dataset, optimizer, device)
    loss.append(loss_value)
    print(f'loss:{loss_value}')
    #scheduler.step()
    valid_result = eval_session(model, valid_dataset, optimizer, device)
    loss_valid = valid_result['loss']
    rouge = valid_result['rouge']
    loss_eval.append(loss_valid)
    rouge_epoch.append(rouge)
    print(f'loss:{loss_valid}')
    print(f'rouge:{rouge}')
print(loss)
print(rouge_epoch)

epoch:0


training....:   0%|          | 0/5000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
training....: 100%|██████████| 5000/5000 [18:51<00:00,  4.42it/s]


loss:5.561052700519562


evaluate.....: 100%|██████████| 1000/1000 [38:13<00:00,  2.29s/it]


loss:2.9463407665491106
rouge:0.11916183315241602
epoch:1


...

loss:1.1030889603048564
rouge:0.18123416462598196
epoch:4


training....: 100%|██████████| 5000/5000 [18:42<00:00,  4.46it/s]


loss:1.6216110142558813


evaluate.....: 100%|██████████| 1000/1000 [55:00<00:00,  3.30s/it]

loss:0.9952329481728375
rouge:0.1851015384325918
[5.561052700519562, 2.8712447287678717, 2.1190885993361475, 1.7970246790647506, 1.6216110142558813]
[0.11916183315241602, 0.16120905457593057, 0.17556242667037172, 0.18123416462598196, 0.1851015384325918]


In [22]:
torch.save(model.state_dict(), '/kaggle/working//state_dict.pt')

In [24]:
model.load_state_dict(torch.load('/kaggle/working//state_dict.pt'))

/tmp/ipykernel_36/2812782820.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working//state_dict.pt'))


<All keys matched successfully>

In [100]:
original_model = model.module
def inference(text):
    inputs = tokenizer.encode(text, return_tensors='pt')
    inputs = torch.tensor(inputs).to(device)
    
    predict_ids = original_model.generate(input_ids = inputs,
                                          num_beams = 5,
                                          early_stopping = True,
                                          eos_token_id=tokenizer.eos_token_id,
                                          do_sample = True,
                                          top_p = 0.9, 
                                          top_k = 40,
                                          max_length = 180)
    print(f'Article:{text}')
    print('--------')
    print(f'Summary:{tokenizer.decode(predict_ids[0])}')

In [101]:
text = dataset['test'][0]['article']
inference(text)

/tmp/ipykernel_36/4017706482.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs).to(device)


Article:(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's cere

In [94]:
text

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o